In [2]:
pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [4]:
pip install langchain_mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.9/415.9 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.44
    Uninstalling langchain-core-0.3.44:
      Successfully uninstalled langchain-core-0.3.44


In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

In [7]:
loader = TextLoader('/content/carboniferous.txt')
docs = loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [9]:
from google.colab import userdata
mistral_api_key = userdata.get('MISTRAL_API_KEY')

In [10]:
embeddings = MistralAIEmbeddings(model="mistral-embed", mistral_api_key=mistral_api_key)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/langchain_mistralai/embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [12]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 54.7 MB/s eta 0:00:00


In [13]:
vector = FAISS.from_documents(documents, embeddings)

In [14]:
retriever = vector.as_retriever()

In [15]:
model = ChatMistralAI(mistral_api_key=mistral_api_key)

In [16]:
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

In [17]:
document_chain = create_stuff_documents_chain(model, prompt)

In [18]:
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [19]:
response = retrieval_chain.invoke({"input": "What is a timeline for carboneferous period?"})

In [20]:
print(response["answer"])

The Carboniferous Period lasted from approximately 359.2 to 299 million years ago. It is further divided into several sub-periods, both in the early and late Carboniferous, based on evolutionary successions of fossil groups. These sub-periods, from early to late, are:

Mississippian (early Carboniferous):
1. Tournaisian
2. Visean
3. Serpukhovian

Pennsylvanian (late Carboniferous):
1. Bashkirian
2. Moscovian
3. Kasimovian
4. Gzhelian

These divisions are used internationally, except in the United States, where the Carboniferous has been separated into the Mississippian and Pennsylvanian Subsystems to account for differing stratigraphy on the different continents.
